In [1]:
import pandas as pd
import numpy as np

In [2]:
app_df=pd.read_csv("./content/cleaned_application.csv",index_col=0)

In [3]:
credit_record=pd.read_csv("./content/credit_record.csv",index_col=0)

C:\Users\siong hwee\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
app_df.head()

,id,gender,own_car,own_realty,num_child,income,income_type,education_level,family_status,house_type,birth_day,employment_length,mobile,work_phone,phone,email,job,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,High skill tech staff,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,High skill tech staff,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [5]:
app_df["mobile"].unique()

array([1], dtype=int64)

Everyone has a mobile phone, we should drop this redundant factor (cardinality=1) to reduce the dimesionality of our data

In [6]:
no_mobile=app_df.drop(columns=["mobile"])

In [7]:
credit_record

,id,month_bal,status
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [8]:
credit_record["month_bal"].unique(),credit_record["status"].unique()

(array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
        -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
        -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
        -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
        -52, -53, -54, -55, -56, -57, -58, -59, -60], dtype=int64),
 array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object))

We could classify '0', 'C', 'X' as good and '1-5' as bad.
We assign C as 0 and X as -1 and map the numerical values using f(x) = x+1.
We can take a mean of the records and treat x<1 as good and x>=1 as bad where x is the mean of each id.
This would be our baseline labeling of the data

In [9]:
d={'C':0,'X':-1}
for i in range(6):
    d[str(i)]=i+1

In [10]:
credit_record["num_status"]=credit_record["status"].apply(lambda x:d[x]) 

In [11]:
labels_df=credit_record.groupby(['id'], as_index=False).mean()

In [12]:
labels_df["good"]=labels_df["num_status"]<1

In [13]:
labels_df=labels_df.drop(labels=["month_bal","num_status"],axis=1)

# To Do
There are some ids not in the application data

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_validate, KFold, cross_val_score

Data preprocessing 

In [16]:
cat_features = df.columns[df.dtypes == 'object'].to_list()
num_features = df.columns[df.dtypes != 'object'].to_list()
num_features.remove('id')

In [17]:
df1 = df.drop(['id'], axis=1).copy()